In [1]:
username = "Alex"
pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")
include(pathtorepo * "dynamical-systems\\system.jl")


using StaticArrays, DifferentialEquations, DynamicalSystems, JLD
E, x, y  = -40..40, -5..5,  -5..5
box = E × x × y
using GLMakie
using LinearAlgebra, LinearSolve

cd("C:\\Users\\" *username *  "\\Desktop\\dynamical-systems\\Tsodyks Markram\\Levanova\\3 набор параметров\\Сопоставление с матконт\\файлы matlab")
I0_hom = load("I0_hom_hom.jld")["data"]
u0_hom = load("U0_hom_hom.jld")["data"]
I0_hom = I0_hom[:]
u0_hom = u0_hom[:];

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\integrate`

In [2]:
@inbounds function Res(u, p, t)
    du1 = -u[2]-u[3]
    du2 = u[1]+p[1]*u[2]
    du3 = p[2]*u[1]-p[3]*u[3]+u[1]*u[3]
    return SVector(du1, du2, du3)
end
@inbounds function jac_res(u, p, t)
    SMatrix{3, 3}(0.0, 1.0, p[2]+u[3],
                -1.0, p[1], 0.0,
                -1.0, 0.0, -p[3]+u[1])
end
function makeevent(fp, ϵbox)
    function condition(u, t, integrator)
        x = Vector(u)
        b = x - fp[1]
        b = Vector(b)
        linprob = LinearProblem(A, b)
        linsolve = solve(linprob)
        return norm(linsolve.u, Inf) - ϵbox
    end
    return condition
end
affect!(integrator) = terminate!(integrator)

function get_norm_αs(u)
    x = Vector(u)
    b = x - fp[1]
    b = Vector(b)
    linprob = LinearProblem(A, b)
    linsolve = solve(linprob)
    return norm(linsolve.u, Inf), linsolve
end

get_norm_αs (generic function with 1 method)

In [3]:
b = 0.3;  c = 3.458; a = 0.4916
t = 1000.0; tspan = [0.0, t]
p = [a, b, c]
u0 = [0.0, 0.0, 0.0];

In [4]:
ds = CoupledODEs(Res, u0, p)
fp, _, _ = fixedpoints(ds, box, jac_res);

In [5]:
ϵ_box = 1e-4
Ju0 = jac_res(fp[1], p, 0);
fp_ = fp[1]

eigen_val_vec = eigen(Ju0)
eigen_vectors = eigen_val_vec.vectors

v1 = real(eigen_vectors[:, 1])
v2 = real(eigen_vectors[:, 2])
v3 = imag(eigen_vectors[:, 3])

v1 = normalize(v1, 2)
v2 = normalize(v2, 2)
v3 = normalize(v3, 2)

v1t = transpose(v1)
v2t = transpose(v2)
v3t = transpose(v3)

A = transpose([v1t; v2t; v3t])
A = Matrix(A)

3×3 Matrix{Float64}:
  0.264692   0.960091    0.0
 -0.068448  -0.269859   -0.999772
  0.961901   0.0734972  -0.0213429

In [6]:
N = 50
nrange = range(1, N - 1, step = 1)
dots = zeros(length(nrange), 3);

In [7]:
α1su0 =  Float64[]; α2su0 =  Float64[]; α3su0 =  Float64[]

for (i, n) in enumerate(nrange)
    α1 = 0
    α2 = cos(2 * pi * n / N) * (ϵ_box * sqrt(2))
    α3 = sin(2 * pi * n / N) * (ϵ_box * sqrt(2))
    αs = [α1, α2, α3]
    dot = fp_ + A * αs
    dots[i, :] = dot
    push!(α2su0, α2)
    push!(α3su0, α3)
end

In [8]:
indexn = 13
condition = makeevent(fp, ϵ_box)
u0 = SA[dots[indexn, 1], dots[indexn, 2], dots[indexn, 3]]
cb = ContinuousCallback(condition, nothing, affect!)
prob = ODEProblem(Res, u0, tspan, p)
sol = solve(prob, alg = Vern9(), abstol = 1e-14, reltol = 1e-14, callback = cb)

retcode: Success
Interpolation: specialized 9th order lazy interpolation
t: 21726-element Vector{Float64}:
    0.0
    0.046760745932695864
    0.13855977142310053
    0.3019344770488754
    0.49584866413229933
    0.7016376031370021
    0.9256487274661314
    1.1603928589990988
    1.4065426222753143
    1.6608807193612327
    ⋮
  999.7422950910174
  999.7757670487954
  999.8090901936666
  999.8422460395167
  999.8753844583641
  999.9086651611678
  999.9421046412682
  999.9757236985665
 1000.0
u: 21726-element Vector{SVector{3, Float64}}:
 [-8.525530802474061e-6, -0.00013871382120750626, -3.6650311541773147e-6]
 [-1.7969128487954162e-6, -0.00014218474799721325, -3.1836612732250496e-6]
 [1.179026930555527e-5, -0.00014828757936786728, -2.1922856367608403e-6]
 [3.694670532242508e-5, -0.000156586676121595, -2.927240264723537e-7]
 [6.771516591472217e-5, -0.00016165977982848047, 2.13900656200935e-6]
 [0.0001002859447993502, -0.00016072506651860298, 4.847810068480159e-6]
 [0.0001340203281512

In [9]:
ts = 1
idx, idy, idz = 2, 3, 1

f = Figure(resolution = (700, 700))
ax = LScene(f[1, 1], show_axis = true)
lines!(sol[idx, ts: length(sol)], sol[idy, ts: length(sol)], sol[idz, ts: length(sol)], linewidth = 1.0, color = :deeppink)
scatter!(ax, fp[1][idx], fp[1][idy], fp[1][idz], color = :blue)
scatter!(ax, dots[:, idx], dots[:, idy], dots[:, idz], markersize = 5)

display(GLMakie.Screen(), f);

In [10]:
f = Figure(resolution = (700, 700))

ax = Axis(f[1, 1])
idx, idy = 2, 3

scatter!(ax, 0.0, 0.0, color = :green, markersize = 15, marker = :rect)

scatter!(ax, linsolve[idx], linsolve[idy], color = :red, markersize = 5)
scatter!(ax, α2su0, α3su0, color = :black, markersize = 5)

display(GLMakie.Screen(), f);

UndefVarError: UndefVarError: `linsolve` not defined